In [ ]:
#https://www.cnblogs.com/pinard/p/6293298.html

In [1]:
from numpy import *

In [2]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [3]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [4]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        #print(transaction)
        for item in transaction:
            #print(item)
            if not [item] in C1:
                #print([item])
                C1.append([item])
                #print(C1)
                
    C1.sort()
    return list(map(frozenset, C1))

In [5]:
C1 = createC1(dataSet)

In [6]:
def scanD(dataSet, Ck, minSupport):
    ssCnt = {}
    for tid in dataSet:
        for can in Ck:
            if can.issubset(tid): #判断can是否是tid的子集
                if can not in ssCnt.keys(): 
                    ssCnt[can]=1
                else: 
                    ssCnt[can] += 1
    numItems = float(len(dataSet))
    retList = []      #频繁项集
    supportData = {}  #候选集项Ck的支持度（k：候选项，value：支持度）
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [7]:
L1, supportData = scanD(dataSet, C1, 0.5)

In [8]:
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [ ]:
# def aprioriGen(Lk, k): #Lk:频繁项集， k=项的个数
#     retList = []
#     lenLk = len(Lk)
#     for i in range(lenLk):
#         for j in range(i+1, lenLk): 
#             C = Lk[i] | Lk[j]
#             if C not in retList and (len(C) == Lk[0]+1)
#                 retList.append(C)
#     return retList

In [9]:
def aprioriGen(Lk, k): #Lk:频繁项集， k=项的个数
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1==L2: #如果L1和L2的前k-2个元素相同
                retList.append(Lk[i] | Lk[j]) #对其直接做并集处理
    return retList

In [10]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    L1, supportData = scanD(dataSet, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(dataSet, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [11]:
L, supportData = apriori(dataSet, minSupport = 0.5)

In [12]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7): #(freqSet：单个频繁项集, H：关联规则右部元素, supportData：支持度, brl：强关联规则列表, minConf：最小置信度)
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [13]:
freqSet = L[2][0]
freqSet

frozenset({2, 3, 5})

In [14]:
brl = []
H = [frozenset([item]) for item in freqSet]
calcConf(freqSet, H, supportData, brl, minConf=0.5)

frozenset({3, 5}) --> frozenset({2}) conf: 1.0
frozenset({2, 5}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({2, 3}) --> frozenset({5}) conf: 1.0


[frozenset({2}), frozenset({3}), frozenset({5})]

In [15]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [16]:
rulesFromConseq(freqSet, H, supportData, brl, minConf=0.1)

frozenset({5}) --> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf: 0.6666666666666666


In [17]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

In [ ]:
brl = generateRules(L, supportData, minConf=0.7)

In [ ]:
import pandas as pd

In [ ]:
mushDataSet=[line.split() for line in open('mushroom.dat').readlines()]

In [ ]:
mushDataSet[0]

In [ ]:
L, supportData=apriori(mushDataSet,minSupport = 0.3)

In [ ]:
L[8]

In [ ]:
L0 = []
for i in range(len(L)):
    for item in L[i]:
        if item.intersection('1'):
            L0.append(item)

In [ ]:
len(L0)

In [ ]:
re = []
for i in range(len(L0)):
    for item in L0[i]:
        if item not in re:
            re.append(item)

In [ ]:
re

In [ ]:
rules = generateRules(L, supportData, minConf=0.95)

In [ ]:
len(rules)

In [ ]:
for i in range(len(rules)):
    if frozenset({'1'}) in rules[i]:
        print(rules[i])

In [ ]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7): #(freqSet：单个频繁项集, H：关联规则右部元素, supportData：支持度, brl：强关联规则列表, minConf：最小置信度)
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))

In [ ]:
brl = []
for i in range(1, len(L0)):#only get the sets with two or more items
    conf = supportData[L0[i]]/supportData[L0[i]-frozenset({'1'})]
    if conf >= 0.95: 
        print(L0[i]-frozenset({'1'}),'-->',frozenset({'1'}),'conf:',conf)
        brl.append((L0[i]-frozenset({'1'}), frozenset({'1'}), conf))